In [ ]:
import os
os.environ["HF_DATASETS_DISABLE_TORCHCODEC"] = "1"

In [1]:
import torch 
torch.__version__

'2.8.0+cu128'

In [2]:
torch.cuda.is_available()

True

In [1]:
from diarizers import SyntheticDataset, SyntheticDatasetConfig

/home/lamhaourii/Documents/2a/audio/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/lamhaourii/Documents/2a/audio/.venv/lib/python3.9/site-packages/pyannote/audio/core/io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()
/home/lamhaourii/Documents/2a/audio/.venv/lib/python3.9/site-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has be

In [ ]:
from diarizers import SyntheticDataset, SyntheticDatasetConfig

segments_per_meeting = 20 
num_meetings = 2000


synthetic_dataset_config = SyntheticDatasetConfig(
    dataset_name="igitsml/darija-speakers-clips",
    subset="train",
    split="default",
    speaker_column_name="label",
    audio_column_name="audio",
    
    min_samples_per_speaker=100,
    nb_speakers_from_dataset=19,  
    sample_rate=16000,
    num_meetings=num_meetings,  
    nb_speakers_per_meeting=2,  
    segments_per_meeting=segments_per_meeting,  
    normalize=True,
    overlap_proba=0.4,  
    overlap_length=1.5,  
    random_gain=True,
    add_silence=True,
    silence_duration=2.0,  
    silence_proba=0.4,  
    denoise=False,
    num_proc=3
)

# Generate dataset
try:
    synthetic_dataset = SyntheticDataset(synthetic_dataset_config).generate()
    print("Synthetic dataset generation complete")
except Exception as e:
    
    print(f"Error during synthetic dataset generation: {e}")
    import traceback
    traceback.print_exc()

nb speakers in dataset to keep: 19


100%|██████████| 19/19 [00:00<00:00, 67938.43it/s]


Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /home/lamhaourii/.cache/torch/hub/master.zip


Map (num_proc=3): 100%|██████████| 39960/39960 [17:23<00:00, 38.31 examples/s]


Synthetic dataset generation complete!


In [3]:
synthetic_dataset

Dataset({
    features: ['audio', 'speakers', 'timestamps_start', 'timestamps_end'],
    num_rows: 1998
})

In [ ]:
from dotenv import dotenv
load_dotenv()
hf_token=os.getenv("hf_token")

In [ ]:
from huggingface_hub import login
login(token=hf_token)

In [6]:
synthetic_dataset.push_to_hub("igitsml/darija-synthetic-calls")

Creating parquet from Arrow format: 100%|██████████| 5/5 [00:01<00:00,  4.22ba/s]
Processing Files (1 / 1): 100%|██████████|  450MB /  450MB,  650kB/s  
New Data Upload: 100%|██████████|  450MB /  450MB,  650kB/s  
Creating parquet from Arrow format: 100%|██████████| 5/5 [00:01<00:00,  4.49ba/s]
Processing Files (0 / 1): 100%|█████████▉|  449MB /  449MB,  340kB/s  
New Data Upload: 100%|██████████|  449MB /  449MB,  340kB/s  
Creating parquet from Arrow format: 100%|██████████| 5/5 [00:01<00:00,  4.19ba/s]
Processing Files (0 / 1): 100%|█████████▉|  442MB /  442MB, 1.61MB/s  
New Data Upload: 100%|██████████|  442MB /  442MB, 1.61MB/s  
Creating parquet from Arrow format: 100%|██████████| 5/5 [00:01<00:00,  3.21ba/s]
Processing Files (1 / 1): 100%|██████████|  445MB /  445MB, 1.06MB/s  
New Data Upload: 100%|██████████|  444MB /  444MB, 1.06MB/s  
Creating parquet from Arrow format: 100%|██████████| 5/5 [00:01<00:00,  2.86ba/s]
Processing Files (1 / 1): 100%|██████████|  446MB /  446MB

CommitInfo(commit_url='https://huggingface.co/datasets/igitsml/darija-synthetic-calls/commit/8e466e41d05e5d9f5308ece3734f96a58317f6b6', commit_message='Upload dataset', commit_description='', oid='8e466e41d05e5d9f5308ece3734f96a58317f6b6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/igitsml/darija-synthetic-calls', endpoint='https://huggingface.co', repo_type='dataset', repo_id='igitsml/darija-synthetic-calls'), pr_revision=None, pr_num=None)

In [7]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Jan 11 02:32:45 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.95.05              Driver Version: 580.95.05      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2060 ...    Off |   00000000:01:00.0 Off |                  N/A |
| N/A   48C    P8              4W /   65W |       4MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----